**PART 1** 

Let's load, clean and explore the data before making the recommender system : 



In [1]:
import pandas as pd

df = pd.read_csv("movies_metadata.csv")

df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Let's create a small_df that only has the following features: title, release_date, budget,
revenue, runtime, and genres :

In [2]:
mycolumns = ["title", "release_date", "budget", "revenue", "runtime", "genres"]

small_df = df[mycolumns].copy()

small_df.head()

,title,release_date,budget,revenue,runtime,genres
0,Toy Story,1995-10-30,30000000,373554033.0,81.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,Jumanji,1995-12-15,65000000,262797249.0,104.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,Grumpier Old Men,1995-12-22,0,0.0,101.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,Waiting to Exhale,1995-12-22,16000000,81452156.0,127.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,Father of the Bride Part II,1995-02-10,0,76578911.0,106.0,"[{'id': 35, 'name': 'Comedy'}]"


Let's check out the data types of the different features : 

In [3]:
print(small_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         45460 non-null  object 
 1   release_date  45379 non-null  object 
 2   budget        45466 non-null  object 
 3   revenue       45460 non-null  float64
 4   runtime       45203 non-null  float64
 5   genres        45466 non-null  object 
dtypes: float64(2), object(4)
memory usage: 2.1+ MB
None


Let's convert every number field in budget to float and, if that fails, convert it to NaN : 

In [5]:
def to_float(number):
  try:
    return float(number)
  except ValueError:
    return float("nan")

Let's apply our to_float function to all values in the budget column, then let's try to convert to float using
pandas astype :

In [6]:
small_df["budget"] = small_df["budget"].apply(to_float)

print(small_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         45460 non-null  object 
 1   release_date  45379 non-null  object 
 2   budget        45463 non-null  float64
 3   revenue       45460 non-null  float64
 4   runtime       45203 non-null  float64
 5   genres        45466 non-null  object 
dtypes: float64(3), object(3)
memory usage: 2.1+ MB
None


Let's define a new feature called “year” that represents the year of release by using the datetime
functionality from pandas as following:

o Convert release_date into pandas datetime format.

o Extract year from the datetime.

In [7]:
small_df["release_date"] = pd.to_datetime(small_df["release_date"], errors="coerce")

In [8]:
small_df["year"] = pd.DatetimeIndex(small_df['release_date']).year

print(small_df.head())

                         title release_date      budget      revenue  runtime  \
0                    Toy Story   1995-10-30  30000000.0  373554033.0     81.0   
1                      Jumanji   1995-12-15  65000000.0  262797249.0    104.0   
2             Grumpier Old Men   1995-12-22         0.0          0.0    101.0   
3            Waiting to Exhale   1995-12-22  16000000.0   81452156.0    127.0   
4  Father of the Bride Part II   1995-02-10         0.0   76578911.0    106.0   

                                              genres    year  
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...  1995.0  
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...  1995.0  
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...  1995.0  
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  1995.0  
4                     [{'id': 35, 'name': 'Comedy'}]  1995.0  


Let's see what's the oldest movies available in this dataset : 

In [9]:
print(small_df.sort_values(by=['year']).head())

                             title release_date  budget  revenue  runtime  \
34940             Passage of Venus   1874-12-09     0.0      0.0      1.0   
34937   Sallie Gardner at a Gallop   1878-06-14     0.0      0.0      1.0   
41602              Buffalo Running   1883-11-19     0.0      0.0      1.0   
34933  Man Walking Around a Corner   1887-08-18     0.0      0.0      1.0   
34934             Accordion Player   1888-01-01     0.0      0.0      1.0   

                                    genres    year  
34940  [{'id': 99, 'name': 'Documentary'}]  1874.0  
34937  [{'id': 99, 'name': 'Documentary'}]  1878.0  
41602  [{'id': 99, 'name': 'Documentary'}]  1883.0  
34933  [{'id': 99, 'name': 'Documentary'}]  1887.0  
34934  [{'id': 99, 'name': 'Documentary'}]  1888.0  


The most successful (revenue feature) movies of all time :

In [10]:
print(small_df.sort_values(by=['revenue'], ascending = False).head())

                              title release_date       budget       revenue  \
14551                        Avatar   2009-12-10  237000000.0  2.787965e+09   
26555  Star Wars: The Force Awakens   2015-12-15  245000000.0  2.068224e+09   
1639                        Titanic   1997-11-18  200000000.0  1.845034e+09   
17818                  The Avengers   2012-04-25  220000000.0  1.519558e+09   
25084                Jurassic World   2015-06-09  150000000.0  1.513529e+09   

       runtime                                             genres    year  
14551    162.0  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...  2009.0  
26555    136.0  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...  2015.0  
1639     194.0  [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...  1997.0  
17818    143.0  [{'id': 878, 'name': 'Science Fiction'}, {'id'...  2012.0  
25084    124.0  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...  2015.0  


Let's create a new DataFrame new of only movies that earned more than $1 billion :

In [11]:
new = small_df[small_df["revenue"] > 1000000000]

print(new.head())

                                               title release_date  \
1639                                         Titanic   1997-11-18   
7000   The Lord of the Rings: The Return of the King   2003-12-01   
11008     Pirates of the Caribbean: Dead Man's Chest   2006-06-20   
12481                                The Dark Knight   2008-07-16   
14551                                         Avatar   2009-12-10   

            budget       revenue  runtime  \
1639   200000000.0  1.845034e+09    194.0   
7000    94000000.0  1.118889e+09    201.0   
11008  200000000.0  1.065660e+09    151.0   
12481  185000000.0  1.004558e+09    152.0   
14551  237000000.0  2.787965e+09    162.0   

                                                  genres    year  
1639   [{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...  1997.0  
7000   [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...  2003.0  
11008  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...  2006.0  
12481  [{'id': 18, 'name': 'Drama'}, {'id': 28

Let's create another new DataFrame new2 of only movies earned more than $1 billion, but where the
budget less than $150 million :

In [12]:
new2 = new[new["budget"] < 150000000]

print(new2)


                                               title release_date  \
7000   The Lord of the Rings: The Return of the King   2003-12-01   
17437   Harry Potter and the Deathly Hallows: Part 2   2011-07-07   
30700                                        Minions   2015-06-17   
44009                                Despicable Me 3   2017-06-15   

            budget       revenue  runtime  \
7000    94000000.0  1.118889e+09    201.0   
17437  125000000.0  1.342000e+09    130.0   
30700   74000000.0  1.156731e+09     91.0   
44009   80000000.0  1.020063e+09     96.0   

                                                  genres    year  
7000   [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...  2003.0  
17437  [{'id': 10751, 'name': 'Family'}, {'id': 14, '...  2011.0  
30700  [{'id': 10751, 'name': 'Family'}, {'id': 16, '...  2015.0  
44009  [{'id': 28, 'name': 'Action'}, {'id': 16, 'nam...  2017.0  


**PART 2**

Building a first simple recommender is straightforward. The steps are as follows:
1. Choose a metric (or score) to rate the movies on ⁉

**--> WR (Weighet Rating)** 

2. Decide on the prerequisites for the movie to be featured on the chart :

**- Runtime : < 45 minutes and < 300 minutes**

**- Number of votes (m) : more than 80% than other movies**

3. Calculate the score for every movie that satisfies the conditions.
4. Output the list of movies in decreasing order of their scores.

**-------------------------------------------------------------**

1. Calculate the weighted rating score.
2. Add the score as a feature to the dataframe, by using Apply function.
3. Sort the dataframe on the basis of the score just computed.
4. Output the list of top movies. 

In [13]:
import numpy as np

#Creating a df with the run time pre-requisite:

mycolumns_2 = ["title", "runtime", "vote_average", "vote_count"]

rating_df_base = df[mycolumns_2].copy()
rating_df_filtered_45 = rating_df_base[rating_df_base["runtime"] > 45]
rating_df_filtered_300 = rating_df_filtered_45[rating_df_filtered_45["runtime"] < 300]

#Calcul of the 80th percentile : 

vote_count = rating_df_filtered_300["vote_count"].to_numpy()
quantile_80th = np.quantile(vote_count, 0.8)

#Creating a rating_df with all the pre-requisite :
rating_df = rating_df_filtered_300[rating_df_filtered_300["vote_count"] > quantile_80th]


In [14]:
#Find the C variable (mean rating of all the movies after filtering)

C = rating_df["vote_average"].mean()
print("Average rate for the movies is : " , C)

#Create the m variable (minimum votes required) : 
m = quantile_80th
print("The minimum of votes required is egal to :", m)

Average rate for the movies is :  6.363846153846154
The minimum of votes required is egal to : 56.0


In [15]:
v = rating_df["vote_count"]
R = rating_df["vote_average"]

val_1 = []
calc_2 = []
val_2 = []

#We now have all the variable, let's calcul the WR Score : 

for i in v:
  calc_1 = (i/i+m)

for i in R:
  val_1.append(calc_1 * i)

for i in v:
  calc_2.append(m/i+m)

for i in calc_2:
  val_2.append(i *C)

from itertools import zip_longest
 
def somme(tup):
    res = 0
    for n in tup:
        res += float(n)
    return res
 
WRScore = []

for val in zip_longest(val_1, val_2, fillvalue = '0'):
    WRScore.append(somme(val))

WRScore_df = pd.DataFrame(WRScore)
WRScore_df.columns = ["WRScore"]

print(WRScore_df)


         WRScore
0     795.341197
1     749.823074
2     730.749030
3     683.335358
4     795.464343
...          ...
8315  745.372935
8316  660.804635
8317  656.566612
8318  586.509369
8319  655.233146

[8320 rows x 1 columns]


In [16]:
rating_df.insert(4, "WRScore", WRScore_df, allow_duplicates=True)

In [17]:
print(rating_df)

                                                title  runtime  vote_average  \
0                                           Toy Story     81.0           7.7   
1                                             Jumanji    104.0           6.9   
2                                    Grumpier Old Men    101.0           6.5   
4                         Father of the Bride Part II    106.0           5.7   
5                                                Heat    170.0           7.7   
...                                               ...      ...           ...   
45265                                   Force Majeure    118.0           6.8   
45268  Good Guys Go to Heaven, Bad Guys Go to Pattaya    100.0           5.3   
45339                                  With Open Arms     92.0           5.2   
45343                      The Visitors: Bastille Day    110.0           4.0   
45441                                     Cadet Kelly    101.0           5.2   

       vote_count     WRScore  
0      

In [18]:
print(rating_df.sort_values(by=['WRScore'], ascending = False).head())

                                               title  runtime  vote_average  \
10309                    Dilwale Dulhania Le Jayenge    190.0           9.1   
7643                              Sansho the Bailiff    124.0           8.7   
29183  The Jinx: The Life and Deaths of Robert Durst    240.0           8.6   
33420                                          Human    263.0           8.6   
40251                                     Your Name.    106.0           8.5   

       vote_count     WRScore  
10309       661.0  875.614530  
7643         68.0  857.516199  
29183        85.0  850.768036  
33420        98.0  850.211868  
40251      1030.0  841.221380  


**PART 3**

1. Using the data in ratings_small.csv, let's construct the user rating matrix (where the rows represent the
users, and the columns the movies) :

In [19]:
df_rating = pd.read_csv("ratings_small.csv")

print(df_rating.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None


In [20]:
matrix = df_rating.pivot(index="userId", columns="movieId", values="rating")
matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2. This matrix will be sparse (lot of unknown values), explain why. Propose a way to fill in the missing
values and justify it. Implement it to have a matrix without missing values.

**There is a lot of unknown values because a lot of user didn’t see/rate all the movies.** 

**The solution that I have chosen is to fill those missing values by the mean rate of each movies**

In [21]:
for i in range(0,9066):
  matrix.iloc[:, i] = matrix.iloc[:, i].replace(float("nan"), matrix.iloc[:,i].mean())

In [22]:
matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,3.87247,3.401869,3.161017,2.384615,3.267857,3.884615,3.283019,3.8,3.15,3.45082,...,2.5,0.5,3.0,1.0,1.5,5.0,4.5,5.0,3.0,5.0
2,3.87247,3.401869,3.161017,2.384615,3.267857,3.884615,3.283019,3.8,3.15,4.00000,...,2.5,0.5,3.0,1.0,1.5,5.0,4.5,5.0,3.0,5.0
3,3.87247,3.401869,3.161017,2.384615,3.267857,3.884615,3.283019,3.8,3.15,3.45082,...,2.5,0.5,3.0,1.0,1.5,5.0,4.5,5.0,3.0,5.0
4,3.87247,3.401869,3.161017,2.384615,3.267857,3.884615,3.283019,3.8,3.15,4.00000,...,2.5,0.5,3.0,1.0,1.5,5.0,4.5,5.0,3.0,5.0
5,3.87247,3.401869,4.000000,2.384615,3.267857,3.884615,3.283019,3.8,3.15,3.45082,...,2.5,0.5,3.0,1.0,1.5,5.0,4.5,5.0,3.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,3.87247,3.401869,3.161017,2.384615,3.267857,4.000000,3.283019,3.8,3.15,3.45082,...,2.5,0.5,3.0,1.0,1.5,5.0,4.5,5.0,3.0,5.0
668,3.87247,3.401869,3.161017,2.384615,3.267857,3.884615,3.283019,3.8,3.15,3.45082,...,2.5,0.5,3.0,1.0,1.5,5.0,4.5,5.0,3.0,5.0
669,3.87247,3.401869,3.161017,2.384615,3.267857,3.884615,3.283019,3.8,3.15,3.45082,...,2.5,0.5,3.0,1.0,1.5,5.0,4.5,5.0,3.0,5.0


3. Calculate the cosine similarity for each pair of users, in order to obtain a user-user similarity matrix.

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(matrix, matrix)


In [54]:
#To be sure that it's on the user: 
print(similarity.shape)

print("Here's the user-user similarity matrix :")
similarity = pd.DataFrame(similarity)

similarity



(671, 671)
Here's the user-user similarity matrix :


,0,1,2,3,4,5,6,7,8,9,...,661,662,663,664,665,666,667,668,669,670
0,1.000000,0.999553,0.999716,0.998778,0.999540,0.999586,0.999544,0.999579,0.999740,0.999730,...,0.999662,0.999803,0.999035,0.998291,0.999631,0.999671,0.999781,0.999783,0.999629,0.999607
1,0.999553,1.000000,0.999596,0.998660,0.999427,0.999456,0.999471,0.999436,0.999595,0.999592,...,0.999527,0.999674,0.998904,0.998074,0.999462,0.999476,0.999640,0.999650,0.999441,0.999491
2,0.999716,0.999596,1.000000,0.998832,0.999571,0.999605,0.999602,0.999601,0.999746,0.999762,...,0.999679,0.999838,0.999059,0.998292,0.999728,0.999719,0.999805,0.999834,0.999649,0.999632
3,0.998778,0.998660,0.998832,1.000000,0.998695,0.998725,0.998660,0.998708,0.998843,0.998835,...,0.998768,0.998926,0.998180,0.997489,0.998729,0.998804,0.998907,0.998904,0.998766,0.998752
4,0.999540,0.999427,0.999571,0.998695,1.000000,0.999458,0.999433,0.999437,0.999588,0.999609,...,0.999539,0.999668,0.998941,0.998242,0.999471,0.999527,0.999631,0.999658,0.999495,0.999454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,0.999671,0.999476,0.999719,0.998804,0.999527,0.999575,0.999555,0.999571,0.999718,0.999727,...,0.999642,0.999791,0.999024,0.998304,0.999645,1.000000,0.999780,0.999770,0.999609,0.999583
667,0.999781,0.999640,0.999805,0.998907,0.999631,0.999683,0.999666,0.999672,0.999831,0.999845,...,0.999749,0.999900,0.999135,0.998374,0.999727,0.999780,1.000000,0.999881,0.999737,0.999696
668,0.999783,0.999650,0.999834,0.998904,0.999658,0.999684,0.999674,0.999672,0.999825,0.999834,...,0.999767,0.999904,0.999155,0.998404,0.999729,0.999770,0.999881,1.000000,0.999741,0.999708
669,0.999629,0.999441,0.999649,0.998766,0.999495,0.999539,0.999511,0.999540,0.999667,0.999655,...,0.999606,0.999748,0.998991,0.998252,0.999550,0.999609,0.999737,0.999741,1.000000,0.999549


4. Implement a function to return the n nearest neighbors of a given user in the sense of the calculated
similarity.

In [96]:
from sklearn.neighbors import NearestNeighbors

#For this part we will try to find the nearest neighbors of the user Id n°3

clf = NearestNeighbors(n_neighbors=2)
clf.fit(similarity)

distances, indices = clf.kneighbors(similarity)

indices = pd.DataFrame(indices)
indices

print("User who have the closest similarity with user 3 is")
print("User :", indices.iloc[2,1] + 1)


User who have the closest similarity with user 3 is
User : 215


5. Using the preceding function, implement a function that recommends k movies—at most—for a
given user. Explain how this function works.

Here's the explanation of the code below : 

First step is to ask the id of the user

Then we use the Nearest Neighbor algorithm we created before. It give us the user ID of the user who is the most similar as our user. 

Then we look at the rating of this similar user in the base matrix we created at the beginning of the exercise. We sort this matrix and print only the 5 best rating of the similar user. 

In [170]:
user = int(input("Enter your id n°"))

Enter your id n°453


In [171]:
user = user - 1

In [172]:
new_matrix = df_rating.pivot(index="userId", columns="movieId", values="rating")

In [173]:
nearest = indices.iloc[user,1] + 1

recommandation = new_matrix.iloc[nearest]
recommandation = pd.DataFrame(recommandation)
recommandation.columns = ['Rating']
recommandation = recommandation.sort_values(by=['Rating'], ascending = False).head()

print("Here's the 5 movies we recommend you :")
recommandation.drop("Rating", axis = 1)


Here's the 5 movies we recommend you :


""
movieId
36
2108
2599
2858
2997
